# 一、数据集说明

本项目使用的数据集是：[[AI训练营]目标检测数据集合集](https://aistudio.baidu.com/aistudio/datasetdetail/103743)里的火焰检测。


In [ ]:
# 解压数据集（解压一次即可，请勿重复解压）
!unzip -oq /home/aistudio/data/data103743/objDataset.zip 

In [ ]:
# 拷贝到work下
!cp -r objDataset/fire/ work/

# 二、数据准备


In [ ]:
# 安装PaddleX
!pip install paddlex

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 522kB 13.0MB/s eta 0:00:01
     |████████████████████████████████| 102kB 56.3MB/s ta 0:00:01
     |████████████████████████████████| 215kB 29.7MB/s eta 0:00:01
     |████████████████████████████████| 153kB 72.1MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 14.7MB/s eta 0:00:01
     |████████████████████████████████| 102kB 63.3MB/s ta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278365 sha256=fda8db00a66692657b9654803ea8eb64f425c07c0fcb436dfb0e996217c244cb
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
Successfully built pycocotools
  Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4


In [ ]:
# 划分数据集，7:2:1
!paddlex --split_dataset --format VOC --dataset_dir work/fire --val_value 0.2 --test_value 0.1

Dataset Split Done.
Train samples: 345
Eval samples: 98
Test samples: 49
Split files saved in work/fire


# 三、数据预处理

在训练模型之前，对目标检测任务的数据进行操作，从而提升模型效果。可用于数据处理的API有：
- **Normalize**：对图像进行归一化
- **ResizeByShort**：根据图像的短边调整图像大小
- **RandomHorizontalFlip**：以一定的概率对图像进行随机水平翻转
- **RandomDistort**：以一定的概率对图像进行随机像素内容变换

更多关于数据处理的API及使用说明可查看文档：
[https://paddlex.readthedocs.io/zh_CN/release-1.3/apis/transforms/det_transforms.html](https://paddlex.readthedocs.io/zh_CN/release-1.3/apis/transforms/det_transforms.html)

In [ ]:
from paddlex.det import transforms

# 定义训练和验证时的transforms
# API说明 https://paddlex.readthedocs.io/zh_CN/develop/apis/transforms/det_transforms.html
train_transforms = transforms.Compose([
    transforms.RandomDistort(),
    transforms.RandomExpand(),
    transforms.RandomCrop(),
    transforms.Resize(target_size=608, interp='RANDOM'),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(),
])

eval_transforms = transforms.Compose([
    transforms.Resize(target_size=608, interp='CUBIC'),
    transforms.Normalize(),
])

读取PascalVOC格式的检测数据集，并对样本进行相应的处理。

In [ ]:
import paddlex as pdx

# 定义训练和验证所用的数据集
# API说明：https://paddlex.readthedocs.io/zh_CN/develop/apis/datasets.html#paddlex-datasets-vocdetection
train_dataset = pdx.datasets.VOCDetection(
    data_dir='work/fire',
    file_list='work/fire/train_list.txt',
    label_list='work/fire/labels.txt',
    transforms=train_transforms,
    shuffle=True)

eval_dataset = pdx.datasets.VOCDetection(
    data_dir='work/fire',
    file_list='work/fire/val_list.txt',
    label_list='work/fire/labels.txt',
    transforms=eval_transforms)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


2021-08-12 16:06:42 [INFO]	Starting to read file list from dataset...
2021-08-12 16:06:43 [INFO]	345 samples in file work/fire/train_list.txt
creating index...
index created!
2021-08-12 16:06:43 [INFO]	Starting to read file list from dataset...
2021-08-12 16:06:43 [INFO]	98 samples in file work/fire/val_list.txt
creating index...
index created!


# 四、模型训练

PaddleX目前提供了FasterRCNN和YOLOv3两种检测结构，多种backbone模型。本基线系统以骨干网络为MobileNetV1的YOLOv3算法为例。

In [ ]:
# 初始化模型
# API说明: https://paddlex.readthedocs.io/zh_CN/develop/apis/models/detection.html#paddlex-det-yolov3

# 此处需要补充目标检测模型代码
model = pdx.det.YOLOv3(num_classes=len(train_dataset.labels), backbone='MobileNetV1')

In [ ]:
# 模型训练
# API说明: https://paddlex.readthedocs.io/zh_CN/develop/apis/models/detection.html#id1
# 各参数介绍与调整说明：https://paddlex.readthedocs.io/zh_CN/develop/appendix/parameters.html

# 此处需要补充模型训练参数
model.train(
    num_epochs=100,
    train_dataset=train_dataset,
    train_batch_size=16,
    eval_dataset=eval_dataset,
    learning_rate=0.00025,
    lr_decay_epochs=[60, 80],
    save_dir='output/yolov3_mobilenetv1')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/framework.py:706: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:


2021-08-12 16:06:44 [INFO]	Downloading MobileNetV1_pretrained.tar from http://paddle-imagenet-models-name.bj.bcebos.com/MobileNetV1_pretrained.tar


100%|██████████| 16760/16760 [00:00<00:00, 20952.04KB/s]


2021-08-12 16:06:45 [INFO]	Decompressing output/yolov3_mobilenetv1/pretrain/MobileNetV1_pretrained.tar...
2021-08-12 16:06:48 [INFO]	Load pretrain weights from output/yolov3_mobilenetv1/pretrain/MobileNetV1_pretrained.
2021-08-12 16:06:48 [INFO]	There are 135 varaibles in output/yolov3_mobilenetv1/pretrain/MobileNetV1_pretrained are loaded.
2021-08-12 16:06:54 [INFO]	[TRAIN] Epoch=1/100, Step=2/21, loss=9082.070312, lr=0.0, time_each_step=3.12s, eta=1:51:2
2021-08-12 16:06:55 [INFO]	[TRAIN] Epoch=1/100, Step=4/21, loss=9859.844727, lr=1e-06, time_each_step=1.69s, eta=0:59:54
2021-08-12 16:06:55 [INFO]	[TRAIN] Epoch=1/100, Step=6/21, loss=4357.087891, lr=1e-06, time_each_step=1.22s, eta=0:43:8
2021-08-12 16:06:56 [INFO]	[TRAIN] Epoch=1/100, Step=8/21, loss=3654.176758, lr=2e-06, time_each_step=0.98s, eta=0:34:54
2021-08-12 16:06:57 [INFO]	[TRAIN] Epoch=1/100, Step=10/21, loss=2215.888672, lr=2e-06, time_each_step=0.86s, eta=0:30:17
2021-08-12 16:06:57 [INFO]	[TRAIN] Epoch=1/100, Step=12

100%|██████████| 7/7 [00:04<00:00,  1.46it/s]


...

2021-08-12 16:16:31 [INFO]	[EVAL] Finished, Epoch=60, bbox_map=39.141155 .
2021-08-12 16:16:33 [INFO]	Model saved in output/yolov3_mobilenetv1/best_model.
2021-08-12 16:16:33 [INFO]	Model saved in output/yolov3_mobilenetv1/epoch_60.
2021-08-12 16:16:33 [INFO]	Current evaluated best model in eval_dataset is epoch_60, bbox_map=39.1411546039172
2021-08-12 16:16:36 [INFO]	[TRAIN] Epoch=61/100, Step=2/21, loss=8.698867, lr=2.5e-05, time_each_step=0.44s, eta=0:8:54
2021-08-12 16:16:37 [INFO]	[TRAIN] Epoch=61/100, Step=4/21, loss=7.291191, lr=2.5e-05, time_each_step=0.44s, eta=0:8:53
2021-08-12 16:16:38 [INFO]	[TRAIN] Epoch=61/100, Step=6/21, loss=9.87142, lr=2.5e-05, time_each_step=0.42s, eta=0:8:52
2021-08-12 16:16:38 [INFO]	[TRAIN] Epoch=61/100, Step=8/21, loss=8.220633, lr=2.5e-05, time_each_step=0.42s, eta=0:8:51
2021-08-12 16:16:39 [INFO]	[TRAIN] Epoch=61/100, Step=10/21, loss=13.704815, lr=2.5e-05, time_each_step=0.43s, eta=0:8:50
2021-08-12 16:16:39 [INFO]	[TRAIN] Epoch=61/100, Step=1

100%|██████████| 7/7 [00:03<00:00,  1.90it/s]


2021-08-12 16:19:42 [INFO]	[EVAL] Finished, Epoch=80, bbox_map=49.62634 .
2021-08-12 16:19:45 [INFO]	Model saved in output/yolov3_mobilenetv1/best_model.
2021-08-12 16:19:45 [INFO]	Model saved in output/yolov3_mobilenetv1/epoch_80.
2021-08-12 16:19:45 [INFO]	Current evaluated best model in eval_dataset is epoch_80, bbox_map=49.626340258869966
2021-08-12 16:19:50 [INFO]	[TRAIN] Epoch=81/100, Step=2/21, loss=7.313561, lr=2e-06, time_each_step=0.49s, eta=0:3:4
2021-08-12 16:19:51 [INFO]	[TRAIN] Epoch=81/100, Step=4/21, loss=11.502262, lr=2e-06, time_each_step=0.51s, eta=0:3:3
2021-08-12 16:19:52 [INFO]	[TRAIN] Epoch=81/100, Step=6/21, loss=8.860214, lr=2e-06, time_each_step=0.52s, eta=0:3:2
2021-08-12 16:19:52 [INFO]	[TRAIN] Epoch=81/100, Step=8/21, loss=9.93677, lr=2e-06, time_each_step=0.51s, eta=0:3:1
2021-08-12 16:19:53 [INFO]	[TRAIN] Epoch=81/100, Step=10/21, loss=8.418819, lr=2e-06, time_each_step=0.49s, eta=0:3:0
2021-08-12 16:19:53 [INFO]	[TRAIN] Epoch=81/100, Step=12/21, loss=8.9

100%|██████████| 7/7 [00:03<00:00,  2.15it/s]


2021-08-12 16:22:51 [INFO]	[EVAL] Finished, Epoch=100, bbox_map=52.199731 .
2021-08-12 16:22:53 [INFO]	Model saved in output/yolov3_mobilenetv1/best_model.
2021-08-12 16:22:54 [INFO]	Model saved in output/yolov3_mobilenetv1/epoch_100.
2021-08-12 16:22:54 [INFO]	Current evaluated best model in eval_dataset is epoch_100, bbox_map=52.19973077270447


# 五、预测可视化


In [ ]:
import paddlex as pdx

In [ ]:
model = pdx.load_model("output/yolov3_mobilenetv1/best_model")
result = model.predict("output/yolov3_mobilenetv1/10.jpg")
pdx.det.visualize("output/yolov3_mobilenetv1/10.jpg", result, threshold=0.1, save_dir="output/yolov3_mobilenetv1")

2021-08-12 19:34:13 [INFO]	Model[YOLOv3] loaded.
2021-08-12 19:34:13 [INFO]	The visualized result is saved as output/yolov3_mobilenetv1/visualize_10.jpg
